# Правило Рунге


In [1]:
from math import cos, sin, exp, factorial
import numpy as np
import sympy as sp
import scipy as sc
from scipy.integrate import quad

a1=0.7
b=3.2
alfa=0
betta=1/4
knots=np.array([a1, (a1+b)/2, b])
F=lambda x:(1.3*cos(3.5*x)*exp(2*x/3) + 6*sin(4.5*x)*exp(-x/8) + 5*x)*((x-a1)**(alfa) * (b-x)**(betta))
func= lambda x: 1.3*cos(3.5*x)*exp(2*x/3) + 6*sin(4.5*x)*exp(-x/8) + 5*x

p = lambda x, j: (x-a1)**(alfa) * (b-x)**(betta)*x**j

In [2]:
def newtonCotes(a, b, n=3):
    knots=np.linspace(a, b, n)
    mu=[]
    for i in range(np.size(knots)):
        mu.append(sc.integrate.quad(p, a, b, args = (i)))
    mu=np.array(mu)
    mu= np.array(mu[:, 0])
    
    X=np.zeros((np.size(knots), np.size(knots)))
    for i in range(np.size(knots)):
        for j in range(np.size(knots)):
            X[i, j]=knots[j]**i
        
    A=np.linalg.solve(X, mu)

    integral=0
    for i in range(np.size(knots)):
        integral+=func(knots[i])*A[i]
    return(integral)

In [3]:
def skf(a, b, knots):
    skf=0
    for i in range(np.size(knots)-1):
        skf+=newtonCotes(knots[i], knots[i+1])
    print(skf)
    return(skf)

In [8]:
n=54
m=2
L=2
knots1=np.linspace(a1, b, n)
knots2=np.linspace(a1, b, n*L)

S1=skf(a1,b, knots1)
S2=skf(a1, b, knots2)

#приближенные представления погрешностей СКФ
R1=(S2-S1)/(1-L**(-m))
R2=(S2-S1)/(L**m-1)

print(R1, R2)

18.744089947407833
18.744090395855945
5.979308165630451e-07 1.4948270414076129e-07


In [5]:
# оптимальные h с требуемой точностью
eps=10**(-6)
h_opt=(b-a1)/n*((eps/abs(R1)))**(1/m)
n_opt=int((b-a1)//h_opt)
print(h_opt,n_opt)

0.0598715884260519 41


## Ричардсона

In [6]:
h=np.zeros(4)
S=np.zeros(4)
h[0]=n
S[0]=-skf(a1, b, np.linspace(a1, b, h[0]))

for i in range(1,4):
    h[i]=h[i-1]*L
    S[i]=-skf(a1, b, np.linspace(a1, b, h[i]))
print(h, S) 

H=np.zeros((4, 4))
for i in range(4):
    H[:,i ]=-1
for i in range(0, 4):
    for j in range(1, 4):
        H[i, j]=(b-a1)/h[i-1]**(j+1)
C=np.linalg.solve(H, S)
print(H)
R1=0
for i in range(1, np.size(C)):
    R1+=C[i]*H[3, i]
print(C[0])
abs(R1)

18.744089947407833
18.744090395855945
18.744090417654693
18.74409041872662
[ 54. 108. 216. 432.] [-18.74408995 -18.7440904  -18.74409042 -18.74409042]
[[-1.00000000e+00  1.33959191e-05  3.10090719e-08  7.17802591e-11]
 [-1.00000000e+00  8.57338820e-04  1.58766448e-05  2.94011941e-07]
 [-1.00000000e+00  2.14334705e-04  1.98458060e-06  1.83757463e-08]
 [-1.00000000e+00  5.35836763e-05  2.48072575e-07  1.14848414e-09]]
18.744089652623884


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  


7.661027396804481e-07